<a href="https://colab.research.google.com/github/arthurcaique/ptfin/blob/analise_textual_arthur/textual_data_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
from os import listdir
from os.path import join as join_path

import pandas as pd

In [0]:
def _get_pet_info(petID: str, file: dict, tab_data: pd.DataFrame):
    entities = [{'entity': entity['name'], 'type': entity['type'], 'salience': 
                 entity['salience']} for entity in file['entities']]
    
    sentences = [{'sentence': sentence['text']['content'], 
                  'sentiment_score': sentence['sentiment']['score'], 
                  'sentiment_magnitude': sentence['sentiment']['magnitude']} 
                 for sentence in file['sentences']]
    
    document_sentiment = file['documentSentiment']
    language = file['language']
    
    pet_infos = tab_data[tab_data['PetID'] 
                         == petID][['AdoptionSpeed', 'Type']].values[0]
    
    adoption_speed = pet_infos[0]
    type_ = pet_infos[1]
    
    return {'entities': entities, 'sentences': sentences, 
            'document_sentiment': document_sentiment,
    'language': language, 'adoption_speed': adoption_speed, 'type': type_}

In [0]:
textual_data_path = '/home/arthur/Documentos/kaggle competitions/pet finder/all/train_sentiment'
tab_data_path = '/home/arthur/Documentos/kaggle competitions/pet finder/all/train.csv'

In [0]:
files = {filename.replace('.json', ''): json.load(open(join_path(textual_data_path, filename)))
         for filename in listdir(textual_data_path)}

In [0]:
tab_data = pd.read_csv(tab_data_path, sep = ',', encoding='utf-8')
tab_data = tab_data[['PetID', 'AdoptionSpeed', 'Type']]

In [0]:
pets_infos = [_get_pet_info(petID=petID, file=file, tab_data=tab_data) for petID, file in files.items()]
pets_infos_df = pd.DataFrame(pets_infos)

In [0]:
pets_infos_df

,adoption_speed,document_sentiment,entities,language,sentences,type
0,2,"{'magnitude': 3.7, 'score': 0.6}","[{'entity': 'boy', 'type': 'PERSON', 'salience...",en,[{'sentence': 'This handsome yet cute boy is u...,1
1,4,"{'magnitude': 0.8, 'score': 0.4}","[{'entity': 'kittens', 'type': 'PERSON', 'sali...",en,[{'sentence': 'Found this 5 kittens and its mu...,2
2,2,"{'magnitude': 2.1, 'score': 0.7}","[{'entity': 'Mummy', 'type': 'OTHER', 'salienc...",en,[{'sentence': 'Mummy and kitties were left at ...,2
3,3,"{'magnitude': 0.1, 'score': 0.1}",[],en,"[{'sentence': 'currently with me', 'sentiment_...",1
4,1,"{'magnitude': 4.2, 'score': 0.8}","[{'entity': 'heart', 'type': 'OTHER', 'salienc...",en,"[{'sentence': 'Thank you, whoever you are, who...",2
5,2,"{'magnitude': 3.3, 'score': 0.1}","[{'entity': 'God', 'type': 'PERSON', 'salience...",en,[{'sentence': 'He was abandon at our doorstep ...,1
6,2,"{'magnitude': 0.1, 'score': 0}","[{'entity': 'puppy', 'type': 'OTHER', 'salienc...",en,[{'sentence': 'The lil' puppy is currently tak...,1
7,3,"{'magnitude': 0.1, 'score': 0.1}","[{'entity': 'Cat', 'type': 'OTHER', 'salience'...",en,"[{'sentence': 'Cat for adoption', 'sentiment_s...",2
8,4,"{'magnitude': 0.1, 'score': 0}","[{'entity': 'health condition', 'type': 'OTHER...",en,"[{'sentence': 'Owned her for 1 year now.', 'se...",2
9,1,"{'magnitude': 2.2, 'score': 0.2}","[{'entity': 'pup', 'type': 'OTHER', 'salience'...",en,[{'sentence': 'Found this vary cute brown and ...,1


In [0]:
#ToDo verificar se os valores das keys nao utilizadas do json sao sempre todas vazias mesmo
#ToDo pesquisar como foi feito o classificacao do score de sentimento das frases e texto
#ToDo analytics: geral, por tipo de pet